## Image Segmentation for Medical diagnosis using PyTorch

In [30]:
import torch
from sklearn.model_selection import train_test_split
import glob
import json
from torch.utils.data import Dataset
import cv2
from torch.utils.data import random_split

### Downlaod the CVC clinic dataset and extract

In [ ]:
# !curl -L -o ~/Downloads/cvcclinicdb.zip\
#   https://www.kaggle.com/api/v1/datasets/download/balraj98/cvcclinicdb
# !mkdir data
# !unzip ~/Downloads/cvcclinicdb.zip -d data

In [6]:
with open("./config.json","r") as f:
    config = json.load(f)
config
image_path = config['data_path']+"Original/"
mask_path  = config['data_path']+"Ground Truth/"

In [13]:
images = glob.glob(image_path+"*"+config['file_extn'])
masks  = glob.glob(mask_path+"*"+config['file_extn'])

In [18]:
"""
Make CVC dataset
"""

class CVCDataset(Dataset):
    def __init__(self, images, transform=None):
        super(CVCDataset, self).__init__()
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img_split = img_path.split('/')
        img_name = img_split[-1]
        img_dir = '/'.join(img_split[:-2])
        mask_path = img_dir+"/Ground Truth/"+img_name
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [35]:
imgset = CVCDataset(images)
trainset, testset = random_split(imgset, [0.9, 0.1])